# Fine-Tuning and Evaluating LLMs with SageMaker Pipelines and MLflow

In [13]:
!pip install sagemaker==2.225.0  datasets==2.18.0 transformers==4.40.0 mlflow==2.13.2 sagemaker-mlflow==0.1.0 --quiet

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import sagemaker
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.function_step import step
from steps.finetune_llama8b_hf import finetune_llama8b
from steps.preprocess_llama3 import preprocess
from steps.eval_mlflow import evaluation
from steps.utils import create_training_job_name
import os

os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd()

## 1. SageMaker Session & IAM Role

In [22]:
import boto3

try:
    role = sagemaker.get_execution_role()
    print(role)
except ValueError:
    iam = boto3.client("iam")
    # Hard coded ARN since, I'm running this notebook locally to reduce AWS costs
    role = "arn:aws:iam::891612587330:role/service-role/AmazonSageMaker-ExecutionRole-20241230T123665"

sess = sagemaker.Session()